### Teymisbestun með MBTI
Sjá [verkefnislýsingu hér](https://haskoliislands.instructure.com/courses/33829/assignments/152512)
og [writing equations in Colab](https://colab.research.google.com/github/EPS-Libraries-Berkeley/volt/blob/main/LaTeX/Equations_and_Formulas.ipynb#:~:text=Getting%20Started%3A%20Basic%20Equations&text=Math%20can%20be%20displayed%20inline,%3D%20z%24%20in%20the%20line.).



## 1. Describe the mathematical programming model

- ***Define the decision variables, constraints, and objective function.***
- ***Explain how the model ensures diversity and balance within the team.***


### **Solution:**

***The mathematical model:***

Our decision variable is a vector $\overline x = (x_1, \dots , x_{16}) \in \mathbb{R}^{16}$ s.t. $x_i$ denotes the number of persons chosen to the team of personality type $i$. We define as well a vector $\overline c \in \mathbb{R}^{16}$ s.t. $c_i$ is the benefit score of personality $i$. As we want to maximize the total benefit score of our team the objective function we want to maximise becomes
$$
\sum_{i=1}^{16} c_ix_i
$$
We define four vectors $a_e, a_a, a_s, a_d \in \mathbb{R}^{16}$ where $a_{e,i}$ is 1 if personality $i$ is an explorer and 0 otherwise. Like wise we define $a_a$ for analysts, $a_s$ for sentinels and $a_d$ diplomats. The dot product $\langle a_a, x\rangle$ then for example returns the total number of analysts. Then we have four constraints:
$$
\langle a_k, x\rangle \geq 1 \text{ for } k\in \{a,e,s,d\}
$$
Then we define vectors $d_r$ where $r\in\{E,I,S,N,T,F,J,P\}$ and e.g. $d_E$ is 1 for all types with Extroversion and 0 for all other types. Then we also have constraints:
$$
-1 \leq \langle d_{r_1} , x\rangle - \langle d_{r_2}, x\rangle \leq 1 \text{ for } (r_1,r_2) \in \{(E,I),(S,N),(T,F),(J,P)\}
$$
The team should consist of 5 to 7 people so we alsi have the constraint
$$
5\leq \sum_{i=1}^{16} x_i \leq 7
$$
Finnally we can only have as many of each personality as are available. If we let $q_i$ denote the number of available persons of personality $i$ we get the constraints
$$
x_i < q_i, \text{ for } i=1, \dots , 16
$$

***Why the team works?***

We have now defined our model. Our main goal is to maximise the benefit score as we want the best team. We also add constraints that each personality group has at most one person and that for each trait we have a no more than 1 person difference. These two constraints ensure we get members of each personality group and personality. With this constraint we are making sure that we do not get members who all have similar talents or expertice. It is also feasible since we only allow as many members of each personality type as are available.


## 2. Formulate and Solve the Linear Program

- Express the LP in matrix form using:
  - **c** (objective function coefficients)
  - **A** (constraint matrix)
  - **b** (right-hand side constraints)
- Solve the LP using x, Table = simplex(c,A,b).

### **Solution:**

We let $c$ be defined as before. Then $A$ will be $30\times 16$ matrix where the first four lines are our $a_k$ vectors, the next both the positive and negative difference of $d_{r_1}$ and $d_{r_1}$ for our four tuples. The next two lines ones and then -1 ones for the team size. Then finally is $A[14:,:]=I_{16}$ to account for the available number of team members. Lastly is then $b$ defined for each constraint. Let us do this in python.

In [2]:
# We begin by defining our personalities
mbti_personality = {
    "Explorers": {
        "ISTP": "Virtuoso",
        "ISFP": "Adventurer",
        "ESTP": "Entrepreneur",
        "ESFP": "Entertainer"
    },
    "Analysts": {
        "INTJ": "Architect",
        "INTP": "Logician",
        "ENTJ": "Commander",
        "ENTP": "Debater"
    },
    "Diplomats": {
        "INFJ": "Advocate",
        "INFP": "Mediator",
        "ENFJ": "Protagonist",
        "ENFP": "Campaigner"
    },
    "Sentinels": {
        "ISTJ": "Logistician",
        "ISFJ": "Defender",
        "ESTJ": "Executive",
        "ESFJ": "Consul"
    }
}

personality_names = {mbti: name for group in mbti_personality.values() for mbti, name in group.items()}
traits = {trait: [mbti for mbti in personality_names.keys() if trait in mbti] for trait in "IESTJFPN"}

# Benefit score for teamwork used for objective function (stuðlar í markfalli)
score = {
    "ESFP": 3.00,
    "ESFJ": 2.93,
    "ENTJ": 2.89,
    "ENTP": 2.81,
    "ENFJ": 2.75,
    "ESTP": 2.75,
    "ENFP": 2.74,
    "ESTJ": 2.72,
    "ISTJ": 2.71,
    "INTJ": 2.61,
    "ISFP": 2.60,
    "INFP": 2.44,
    "ISTP": 2.38,
    "INTP": 2.33,
    "ISFJ": 2.33,
    "INFJ": 2.25,
}
number = {
    "ESFP": 1,
    "ESFJ": 3,
    "ENTJ": 9,
    "ENTP": 6,
    "ENFJ": 2,
    "ESTP": 3,
    "ENFP": 5,
    "ESTJ": 4,
    "ISTJ": 8,
    "INTJ": 8,
    "ISFP": 2,
    "INFP": 3,
    "ISTP": 3,
    "INTP": 1,
    "ISFJ": 1,
    "INFJ": 1,
}


The simplex method

In [3]:
import numpy as np

# Simplex aðferð fyrir max c'x, m.t.t. Ax <= b
def pivot(Table, row, col):
    Table[row, :] /= Table[row, col]
    for k in range(Table.shape[0]):
        if k != row:
            Table[k, :] -= Table[row, :] * Table[k, col]
    return Table

def simplex(c, A, b):
    m, n = A.shape  # Number of constraints (m) and variables (n)

    # If any b is negative then add an artificial x0 variable
    if np.any(b < 0):
      nx = 1
    else:
      nx = 0

    # Initialize simplex tableau
    Table = np.zeros((m+1, n+nx+m+1))
    if nx > 0:
        Table[0, :-1] = np.concatenate((np.zeros(n+m), [1]))  # fake objective
        Table[1:, :-1] = np.hstack((A, np.eye(m), -np.ones((m,1))))  # Constraints (slack vars)
    else:
        Table[0, :-1] = np.concatenate((-c, np.zeros(m)))  # Objective row
        Table[1:, :-1] = np.hstack((A, np.eye(m)))  # Constraints (slack vars)
    Table[1:, -1] = b  # RHS values

    # Initial basis variables (slack variables)
    basis = np.array([n + i for i in range(m)])
    nonbasis = [i for i in range(n + m) if i not in basis]
    iter = 0
    while True:

        # Step 1: Check for special pivot
        if iter == 0 and nx > 0:
          #print("Special pivoting...")
          col = n + m
          row = np.argmin(Table[1:,-1]) + 1

        # Step 2: Check for negative cost coefficients (Primal Feasibility)
        elif np.any(Table[0, :-1] < 0):
            #print("Primal Pivoting...")
            col = np.argmin(Table[0, :-1])  # Most negative reduced cost
            # Standard ratio test to find leaving variable
            ratios = np.array([
                Table[i, -1] / Table[i, col] if Table[i, col] > 0 else np.inf
                for i in range(1, m + 1)
            ])
            row = np.argmin(ratios) + 1  # Get corresponding row

            if np.all(ratios == np.inf):
                raise ValueError("Primal infeasibility detected (no valid leaving row)")

        # Step 3: If neither primal nor dual issues, stop
        else:
            break

        # Perform the pivot
        Table = pivot(Table, row, col)
        basis[row - 1] = col  # Update basis
        nonbasis = [i for i in range(n+m) if i not in basis]
        # if we are using the fake objective and x0 variable check if we can get rid of it
        if (nx > 0) and (Table[0, -1] <= 1E-6) and (n+m) not in basis:
          #print("reinserting the original objective")
          nx = 0
          Table[0,:] = 0
          Table[0,:n] = -c
          Table = np.delete(Table, -2, axis=1)
          for i in range(m):
            Table = pivot(Table,i+1,basis[i])
        iter += 1
    # Extract solution
    if nx > 0:
        x = None
        print("Warning: no feasible solution found")
    else:
        x = np.zeros(n + m)
        x[basis] = Table[1:, -1]

    return x, Table, basis, nonbasis  # Return solution (also slack variables, else use x[:n])

Let us solve the probem

In [7]:
import numpy as np

# Let us define A, b and c and then solve the simplex problem
personalities = list(number.keys())
c = list(score.values())

A = np.zeros((30,16))
b = np.zeros(30)

# At least one from each personality, these are -a_k
A[0,:] = [-1,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0]
b[0] = -1
A[1,:] = [0,0,-1,-1,0,0,0,0,0,-1,0,0,0,-1,0,0]
b[1] = -1
A[2,:] = [0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,-1]
b[2] = -1
A[3,:] = [0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0]
b[3] = -1

#no more than one person difference personality trait axis, these are dr1-dr2 constraints
index=4
for tuple in [['E','I'],['S','N'],['T','F'],['J','P']]:
    r1 = tuple[0]
    r2 = tuple[1]
    dr1 = np.array([1 if r1 in s else 0 for s in personalities])
    dr2 = np.array([1 if r2 in s else 0 for s in personalities])
    delta_r = dr1-dr2
    A[index,:] = delta_r
    b[index] = 1
    index += 1
    A[index,:] = -1 * delta_r
    b[index] = 1
    index += 1

# between 5 and 7 members
A[index,:] = np.ones(16)
b[index] = 7
index +=1
A[index,:] = -1 * np.ones(16)
b[index] = -5
index+=1

# no more than available members allowed
for i in range(16):
    arr = np.zeros(16)
    arr[i] = 1
    A[index+i,:] = arr
    b[index+i] = number[personalities[i]]
 
# solve the LP
x, Table, basis, nonbasis = simplex(np.array(c),np.array(A),np.array(b))
sol = x[:16]

In [5]:
def printResults(sol):
    '''
    A function that prints the solution of our LP
    '''
    total_score = 0
    total_members = 0
    total_types = {
        "Explorers": 0,
        "Analysts": 0,
        "Diplomats": 0,
        "Sentinels": 0
    }
    total_traits = {
        "E":0,
        "I":0,
        "S":0,
        "N":0,
        "F":0,
        "T":0,
        "P":0,
        "J":0
    }
    key_traits = {
        "E":"Extroversion",
        "I":"Introversion",
        "S":"Sensing",
        "N":"Intuition",
        "F":"Feeling",
        "T":"Thinking",
        "P":"Perceiving",
        "J":"Judging"
    }
    print('Team combination:')
    for i in range(len(sol)):
        if(round(sol[i])>0):
            # calc total score
            total_score += round(sol[i])*score[personalities[i]]
            # calc total members
            total_members += round(sol[i])
            # calc total of each type
            for category, types in mbti_personality.items():
                if personalities[i] in types:
                    total_types[category] += round(sol[i])
            # calc totals in each trait
            traits = list(personalities[i])
            for letter in traits:
                total_traits[letter] += round(sol[i])
            print(f'Personality {personalities[i]}, {personality_names[personalities[i]]}, with {round(sol[i])} members')

    print(f'\nTotal score: {round(total_score,2)}')
    print(f'Total members: {total_members}')
    print('\nNumber of members in each MBTI group:')
    for i in total_types.keys():
        print(f'Group {i} with {total_types[i]} members')

    print(f'\nNumbr of members along eack key trait:')
    for i in total_traits.keys():
        print(f'Trait {i}, {key_traits[i]}, has {total_traits[i]} members')


## 3. Sensitivity Analysis (Using the Final Simplex Tableau)

### **a)** *Optimal Team Composition*
- Based on the final Simplex tableau, determine the optimal team composition.
- If fractional values appear in the solution, how should they be interpreted?

#### **Solution**

For fractional values we round to the next whole integer. Then if we use `printResult()` function defined above we get: 

In [8]:
printResults(sol)

Team combination:
Personality ESFP, Entertainer, with 1 members
Personality ENTJ, Commander, with 2 members
Personality ENFP, Campaigner, with 1 members
Personality ISTJ, Logistician, with 2 members
Personality ISFP, Adventurer, with 1 members

Total score: 19.54
Total members: 7

Number of members in each MBTI group:
Group Explorers with 2 members
Group Analysts with 2 members
Group Diplomats with 1 members
Group Sentinels with 2 members

Numbr of members along eack key trait:
Trait E, Extroversion, has 4 members
Trait I, Introversion, has 3 members
Trait S, Sensing, has 4 members
Trait N, Intuition, has 3 members
Trait F, Feeling, has 3 members
Trait T, Thinking, has 4 members
Trait P, Perceiving, has 3 members
Trait J, Judging, has 4 members


### **b)** *Excluded Personality types*
* Identify personality types that did not make it into the team.
* Using the final Simplex tableau, determine how much their contribution would need to increase for them to be included in the team.

#### **Solution:**

We begin by defining what making the team means. In our case we determine that a personality type has made the team if and only if it's included in the final basis whether or not it's final value is zero. Likewise the traits that did not make the team are then the personalities that ended in the non-basis. Then we have:

In [6]:
print('The following personality traits did not make the team')
for i in nonbasis:
        if(i<16):
            print(f'Personality {personalities[i]}, {personality_names[personalities[i]]}')

The following personality traits did not make the team
Personality ESFJ, Consul
Personality ENFJ, Protagonist
Personality ESTP, Entrepreneur
Personality ESTJ, Executive
Personality INFP, Mediator
Personality ISTP, Virtuoso
Personality INTP, Logician
Personality ISFJ, Defender
Personality INFJ, Advocate


Then we can use the reduced costs to determine how much $c_j$ must increase for variable $j$ to enter the basis. If the i-th value in the non-basis is variable $j$ then $z_{n_j}$ is how much $c_j$ must increase s.t. variable $j$ joins the basis.

In [17]:
import sympy as sp
import numpy as np
x, Table, basis, nonbasis = simplex(np.array(c),np.array(A),np.array(b))
m, n = A.shape  # Number of constraints (m) and variables (n)

# Set up the initial Simplex tableau
chat = np.concatenate((c, np.zeros(m)))
Ahat = np.hstack((A, np.eye(m)))

# Compute B and N matrices from the basis
B = Ahat[:, basis]
N = Ahat[:, nonbasis]

# Compute the inverse of B
invB = np.linalg.inv(B)

# Compute B^(-1) * N
invBN = invB @ N
invBN = sp.Matrix(invBN)

cb = sp.Matrix(chat[basis]).T
cn = sp.Matrix(chat[nonbasis]).T

# Compute the reduced costs
zn = invBN.T @ cb.T - cn.T

for i in range(len(nonbasis)):
    if (nonbasis[i] < 16):
        print(f'The parameter c for personality {personalities[nonbasis[i]]} must increase by more than {zn[i]} for the personality to enter the basis')

The parameter c for personality ESFJ must increase by more than 0.0300000000000002 for the personality to enter the basis
The parameter c for personality ENFJ must increase by more than 0.0700000000000003 for the personality to enter the basis
The parameter c for personality ESTP must increase by more than 0.160000000000000 for the personality to enter the basis
The parameter c for personality ESTJ must increase by more than 0.270000000000000 for the personality to enter the basis
The parameter c for personality INFP must increase by more than 0.0200000000000000 for the personality to enter the basis
The parameter c for personality ISTP must increase by more than 0.250000000000000 for the personality to enter the basis
The parameter c for personality INTP must increase by more than 0.200000000000000 for the personality to enter the basis
The parameter c for personality ISFJ must increase by more than 0.350000000000000 for the personality to enter the basis
The parameter c for personali

We can also read the reduced costs straight from the final Simplex Tableau.

In [27]:
def display_final_obj(Table, basis, decision_vars):
    rounded_table = np.round(Table, 4)
    df= pd.DataFrame(rounded_table)
    df.index = ["OBJ"] + [f"{personalities[b]}" if b<16 else f"y_{b}" for b in basis]
    df.columns = [f"col{i}" for i in range(Table.shape[1])]
    if decision_vars and len(decision_vars) <= Table.shape[1]:
        df.columns = decision_vars + [f"y_{i-len(decision_vars)}" for i in range(len(decision_vars), Table.shape[1]-1)] + ['RHS']
    print(df.iloc[[0]].to_string(index=True, header=True))
    print("")
    return df.iloc[[0]]

In [28]:
final_obj = display_final_obj(Table, basis, personalities)

     ESFP  ESFJ  ENTJ  ENTP  ENFJ  ESTP  ENFP  ESTJ  ISTJ  INTJ  ISFP  INFP  ISTP  INTP  ISFJ  INFJ  y_0  y_1   y_2  y_3   y_4  y_5   y_6  y_7    y_8  y_9  y_10  y_11   y_12  y_13  y_14  y_15  y_16  y_17  y_18  y_19  y_20  y_21  y_22  y_23  y_24  y_25  y_26  y_27  y_28  y_29    RHS
OBJ   0.0  0.03   0.0   0.0  0.07  0.16   0.0  0.27   0.0   0.0   0.0  0.02  0.25   0.2  0.35  0.29  0.0  0.0  0.04  0.0  0.14  0.0  0.05  0.0  0.015  0.0  0.04   0.0  2.745   0.0  0.12   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  19.54



And the parameter in the final objective function for non basic variable $j$ is how much it must increase. As an example we can see how the reult changes if we add either add 0.349 to $c_\text{ESTJ}$ or add 0.351.

In [19]:
# We add excactly the reduced cost
add = np.array([0.27 if i == 'ESTJ' else 0 for i in personalities])
c_add = np.array(c)+add
# add the difference
score['ESTJ'] +=  0.27
x, t, bas, nbas = simplex(c_add, np.array(A), np.array(b))
printResults(x[:16])
# remove the differenece again
score['ESTJ'] -=  0.27

Team combination:
Personality ESFP, Entertainer, with 1 members
Personality ENTJ, Commander, with 2 members
Personality ENFP, Campaigner, with 1 members
Personality ISTJ, Logistician, with 2 members
Personality ISFP, Adventurer, with 1 members

Total score: 19.54
Total members: 7

Number of members in each MBTI group:
Group Explorers with 2 members
Group Analysts with 2 members
Group Diplomats with 1 members
Group Sentinels with 2 members

Numbr of members along eack key trait:
Trait E, Extroversion, has 4 members
Trait I, Introversion, has 3 members
Trait S, Sensing, has 4 members
Trait N, Intuition, has 3 members
Trait F, Feeling, has 3 members
Trait T, Thinking, has 4 members
Trait P, Perceiving, has 3 members
Trait J, Judging, has 4 members


In [20]:
# We add a number higher than the reduced costs
add = np.array([0.28 if i == 'ESTJ' else 0 for i in personalities])
c_add = np.array(c)+add
# add the difference to score
score['ESTJ'] +=  0.28
x, t, bas, nbas = simplex(c_add, np.array(A), np.array(b))
printResults(x[:16])
# remove the differenece again
score['ESTJ'] -=  0.28

Team combination:
Personality ESFP, Entertainer, with 1 members
Personality ENFP, Campaigner, with 1 members
Personality ESTJ, Executive, with 2 members
Personality INTJ, Architect, with 2 members
Personality ISFP, Adventurer, with 1 members

Total score: 19.56
Total members: 7

Number of members in each MBTI group:
Group Explorers with 2 members
Group Analysts with 2 members
Group Diplomats with 1 members
Group Sentinels with 2 members

Numbr of members along eack key trait:
Trait E, Extroversion, has 4 members
Trait I, Introversion, has 3 members
Trait S, Sensing, has 4 members
Trait N, Intuition, has 3 members
Trait F, Feeling, has 3 members
Trait T, Thinking, has 4 members
Trait P, Perceiving, has 3 members
Trait J, Judging, has 4 members


We see that this acts like we expect since when we increase $c_\text{ESTJ}$ by more than its reduced costs ESTJ joins the team and if we dont, it stays in the nonbasis.

### **c)** *Find the personality type that just made the team using the final simplex tableu*
* Find the personality type that just made the team using the final Simplex tableau.

* How much must its contribution decrease before it is dropped?

* Which personality type would replace it?


#### **Solution**

When talking abaout which type just made it we mean which variable in the basis has the smallest allowable decrease. Then we calculate the range for $c$ s.t. the optimality of the basic solution is held.

In [10]:
for i in range(len(basis)):
    if basis[i] < 16:
        var_index = basis[i]
        dc = sp.symbols('dc')
        arr = [dc if j == i else 0 for j in range(len(basis))]
        cb = sp.Matrix(chat[basis]).T + sp.Matrix(arr).T
        cn = sp.Matrix(chat[nonbasis]).T
        # Compute the reduced costs
        zn = invBN.T @ cb.T - cn.T

        # Solve for the allowable range
        crange = []
        for i in range(len(c)):
            tmp = zn[i] >= 0
            if tmp != False and tmp != True:
                crange.append(tmp)

        crange = sp.solvers.inequalities.reduce_inequalities(crange)
        print(f'personality {personalities[var_index]} -> {crange}')

personality ENTP -> (-0.0299999999999998 <= dc) & (dc <= 0.0300000000000011)
personality ISFP -> (-0.0300000000000002 <= dc) & (dc <= 0.0299999999999998)
personality ENTJ -> (-0.0150000000000001 <= dc) & (dc <= 0.0199999999999996)
personality ESFP -> (-0.12 <= dc) & (dc < oo)
personality ISTJ -> (-0.0299999999999998 <= dc) & (dc <= 0.04)
personality ENFP -> (-0.02 <= dc) & (dc <= 0.04)
personality INTJ -> (-0.02 <= dc) & (dc <= 0.0300000000000002)


Then we can see that the variable with the smallest allowable decrease is *ENTJ*.

***How much is it's contribution decreased before it is dropped?***

By the ranges calculated from the final simplex tableau shown above we can see that the contribution of variable *ENTJ* must be decreased by more than 0.015 to remove the optimality of the curreent solution.

***Which personality type would replace it?***

When we drop the allowable range the solution becomes non-optimal. We have the condition $$\overline z = (B^{-1}N)^T\overline c_B - \overline c_N\geq 0$$ for the problem to remain optimal. Let us then decrease $c_\text{ENTJ}$ by more than it allowable decrease. 

In [60]:
argInPers = list(personalities).index('ENTJ')
argInBasis = list(basis).index(argInPers)
arr = [-0.0151 if j == argInBasis else 0 for j in range(len(basis))]
cb = sp.Matrix(chat[basis]).T + sp.Matrix(arr).T
cn = sp.Matrix(chat[nonbasis]).T
# Compute the reduced costs
zn = invBN.T @ cb.T - cn.T
print('The reduced costs z_n:')
print(np.array(zn))
print(f'The {np.argmin(zn)} element of z_n has the smallest value which means that variable {personalities[nonbasis[np.argmin(zn)]]} will go into the basis')

The reduced costs z_n:
[[-0.000199999999999534]
 [0.0549000000000004]
 [0.160000000000000]
 [0.254900000000001]
 [0.0350999999999999]
 [0.265100000000000]
 [0.215100000000000]
 [0.334900000000000]
 [0.290000000000000]
 [0.0248999999999997]
 [0.132450000000000]
 [0.0500000000000000]
 [0.0225499999999998]
 [0.0324500000000001]
 [2.73745000000000]
 [0.135100000000000]]
The 0 element of z_n has the smallest value which means that variable ESFJ will go into the basis


When we lower the $c_j$ value for ENTJ by more then the allowable decrease the reduced costs will become as above. Then we detirmined that $z_\text{ESFJ}$ became smallest and negative. Then it joins the basis as ENTJ leaves it. This is compareable to what we learnt about that the value with the smallest negative value in the objective function is chosen to join the basis.

### **d)** Impact of Increasing Lower Bounds

* Using the final Simplex tableau, analyze how increasing the lower bound for Explorers, Analysts, Diplomats, or Sentinels by 1 affects:
    * The overall team composition
    * The total contribution score

#### **Solution:**

Let us begin by writing a function that visualizes the final simplex tableau.

In [61]:
import pandas as pd
def display_simplex_tableau(Table, basis, decision_vars):
    rounded_table = np.round(Table, 4)
    df= pd.DataFrame(rounded_table)
    df.index = ["OBJ"] + [f"{personalities[b]}" if b<16 else f"y_{b-16}" for b in basis]
    df.columns = [f"col{i}" for i in range(Table.shape[1])]
    if decision_vars and len(decision_vars) <= Table.shape[1]:
        df.columns = decision_vars + [f"y_{i-len(decision_vars)}" for i in range(len(decision_vars), Table.shape[1]-1)] + ['RHS']
    print(df.to_string(index=True, header=True))
    print("")
    return df


In [62]:
simplex_tableau = display_simplex_tableau(Table, basis, personalities)

      ESFP  ESFJ  ENTJ  ENTP  ENFJ  ESTP  ENFP  ESTJ  ISTJ  INTJ  ISFP  INFP  ISTP  INTP  ISFJ  INFJ  y_0  y_1   y_2  y_3   y_4  y_5   y_6  y_7    y_8  y_9  y_10  y_11   y_12  y_13  y_14  y_15  y_16  y_17  y_18  y_19  y_20  y_21  y_22  y_23  y_24  y_25  y_26  y_27  y_28  y_29    RHS
OBJ    0.0  0.03   0.0   0.0  0.07  0.16   0.0  0.27   0.0   0.0   0.0  0.02  0.25   0.2  0.35  0.29  0.0  0.0  0.04  0.0  0.14  0.0  0.05  0.0  0.015  0.0  0.04   0.0  2.745   0.0  0.12   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  19.54
ENTP   0.0 -1.00   0.0   1.0 -1.00  1.00   0.0  0.00   0.0   0.0   0.0  0.00  1.00   1.0 -1.00 -1.00  0.0  0.0  0.00  0.0  0.00  0.0  0.00  0.0  0.500  0.0 -0.50   0.0  0.000   0.0 -0.00   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.00
ISFP   0.0  1.00   0.0   0.0 -0.00 -0.00   0.0  0.00   0.0   0.0   1.0  0.00 -0.00   0.0  1.00  0.00  0.0  0.0  1.00  0.0  0.00  0.0  0.00  0.0 -0.5

Firstly we note that $y_0$, $y_1$, $y_2$ and $y_3$ are the slack variables corresponding to the constraints that each personality group must have more than 1 member. We see from the table that their reduced costs are $0$, $0$, $0.04$ and $0$ respectively. These are excactly the shadow prices for $y_0$, $y_1$, $y_2$ and $y_3$. We have:
- The shadow price for $y_0$ is 0
- The shadow price for $y_1$ is 0
- The shadow price for $y_2$ is 0.04
- The shadow price for $y_3$ is 0

The shadow price tells us how much the objective value will increase when the corresponding rhs value is changed. For $y_0$, $y_1$ and $y_3$ the shadow price is 0 so when we increase the minimum amount of members for the corresponding group i.e. decrease the rhs value by one, the objective value doesn't change. This tracks since these slack variables represent groups, Explorers, Analysts and Sentinels and they all have two members so increasing the minimum amount of members there neither affects the team nor the objective value.

The shadow price of $y_2$ which represents Diplomats having at least one member is though positive with the value 0.04. This means that when we increase the minimum amount of Diplomats, decreasing $b_2$ from -1 to -2 the objective function value decreases by 0.04.

We also have the condition $x_B = B^{-1}(b+\Delta b)\geq 0$. Now let us look at the basis values when we decrease $b_2$ by one.

In [120]:
b_delta = [-1 if i == 2 else 0 for i in range(30)]
xb = invB @ (b + b_delta)
print(f'We have x_B = {xb}')
for i in range(30):
    if basis[i] < 16:
        if xb[i] <0:
            print(f'Variable {personalities[basis[i]]} with value {xb[i]} is infeasible')
    else:
        if xb[i] <0:
            print(f'Variable y_{basis[i]-16} with value {xb[i]} is infeasible')

[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
We have x_B = [ 0.  -0.9  0.1  2.9  1.   2.   3.9  2.   2.   2.  -0.9  2.  -0.9  2.9
  0.   3.   8.9  6.   2.   3.   2.1  4.   4.1  8.   2.9  3.   3.   1.
  1.   1. ]
Variable ENFP with value -0.8999999999999999 is infeasible
Variable ISFP with value -0.8999999999999999 is infeasible
Variable y_1 with value -0.8999999999999999 is infeasible


So the values for these three variables break the feasability of our solution so we must remove at least one from the basis and pivot to restore feasibiltiy.

### **e)** Impact of Decreasing Upper Bounds
* What happens when we decide to decrease the allowable number of students in a team down to five? How does the basis solution change?


#### **Solution:**

We note that $y_{12}$ is the slack variable representing the constraint $\sum_{i=0}^{16}x_i \leq 7 = b_{12}$. We note that this constraint is tight since $\sum_{i=0}^{16}=b_{12}$. We can also see from the final tableau that the shadow price is non zero and its value is $2.745$. This means that when we decrease the value of $b_{12}$ from 7 to 5 both the basis must change since the constraint is tight and the objective value will decrease significally (a factor of 2.745). 

Again do we then look at the condition $x_B = B^{-1}(b+\Delta b)\geq 0$. Let us decrease $b_{12}$ by two and observe the basis.

In [124]:
b_delta = [-2 if i == 12 else 0 for i in range(30)]
xb = invB @ (b + b_delta)
print(f'We have x_B = {xb}')
for i in range(30):
    if basis[i] < 16:
        if xb[i] <0:
            print(f'Variable {personalities[basis[i]]} with value {xb[i]} is infeasible')
    else:
        if xb[i] <0:
            print(f'Variable y_{basis[i]-16} with value {xb[i]} is infeasible')

We have x_B = [ 0.  -0.9  0.1  1.9  1.   2.   2.9  2.   0.   2.  -0.9  2.  -0.9  1.9
  0.   3.   8.9  6.   2.   3.   3.1  4.   5.1  8.   2.9  3.   3.   1.
  1.   1. ]
Variable ENFP with value -0.8999999999999999 is infeasible
Variable ISFP with value -0.8999999999999999 is infeasible
Variable y_1 with value -0.8999999999999999 is infeasible


Here we again see which variables break feasibilty and must be pivoted around to try to restore it. It i sinteresting to note that these are excactly the same variables as in d) above. Finally lets perform the simplex algorithm on the new problem to see how the solution changes.

In [125]:
# define b_add to tighten the constraints to be from 3 to 5 instead of 5 to 7
b_add = [-2 if i == 12 else 0 for i in range(30)]
b_add[13] = 2
x_1, table_1, basis_1, nonbasis_1 = simplex(np.array(c), np.array(A),np.array(b)+np.array(b_add))
printResults(x_1[:16])

Team combination:
Personality ESFP, Entertainer, with 1 members
Personality ENFJ, Protagonist, with 1 members
Personality ENFP, Campaigner, with 1 members
Personality ISTJ, Logistician, with 1 members
Personality INTJ, Architect, with 1 members

Total score: 13.81
Total members: 5

Number of members in each MBTI group:
Group Explorers with 1 members
Group Analysts with 1 members
Group Diplomats with 2 members
Group Sentinels with 1 members

Numbr of members along eack key trait:
Trait E, Extroversion, has 3 members
Trait I, Introversion, has 2 members
Trait S, Sensing, has 2 members
Trait N, Intuition, has 3 members
Trait F, Feeling, has 3 members
Trait T, Thinking, has 2 members
Trait P, Perceiving, has 2 members
Trait J, Judging, has 3 members


## 4. Formulate and Interpret the Dual Problem

- Write the dual of the original LP.
- Describe its interpretation in the context of team formation.
    - What do the dual variables represent?
    - How does the dual provide insight into the importance of different constraints?


### **Solution:**

#### Write the dual of the original LP.
Let us begin by defining the primal problem in standard form. We have:

**Primal:**
$$
\underset{x_1, \dots x_{16}}{\max} \zeta = \sum_{j=1}^{16}c_jx_j
$$
such that
$$
\begin{align*}
&\sum_{j=1}^{16}-a_{kj}x_j \leq -1 \quad \text{for all } k\in\{a,e,s,d\}\\
&\sum_{j=1}^{16}d_{r_1j}x_j-d_{r_2j}x_j \leq 1 \quad \text{for all } (r_1,r_2) \in \{(E,I),(S,N),(T,F),(J,P)\}\\
&\sum_{j=1}^{16}d_{r_2j}x_j-d_{r_1j}x_j \leq 1 \quad \text{for all } (r_1,r_2) \in \{(E,I),(S,N),(T,F),(J,P)\}\\
&\sum_{j=1}^{16}x_j \leq 7\\
&\sum_{j=1}^{16}-x_j \leq -5\\
&\sum_{j=1}^{16}e_ix_j \leq q_i \quad \text{for all } i = 1, \dots, 16
\end{align*}
$$
and $x_j\geq 0$ for $j=1,\dots, 16$. From this we have matrix A and vector b
$$A = 
\begin{bmatrix}
-a_a\\
-a_e\\
-a_s\\
-a_d\\
d_E-d_I\\
d_S-d_N\\
d_T-d_F\\
d_J-d_P\\
d_I-d_E\\
d_N-d_S\\
d_F-d_T\\
d_P-d_J\\
\overline 1\\
-\overline 1\\
e_1\\
\vdots \\
e_{16}
\end{bmatrix}, \; b= 
\begin{bmatrix}
-1\\
-1\\
-1\\
-1\\
1\\
1\\
1\\
1\\
1\\
1\\
1\\
1\\
7\\
-5\\
q_1\\
\vdots \\
q_{16}
\end{bmatrix}
$$
Then we can state the dual problem as:

**Dual:**

$$
\underset{y_1,\dots, y_{30} }{\min} \xi  = \sum_{i=1}^{30}b_iy_i
$$

subject to

$$
\sum_{i=1}^{30}y_iA_{ij} \geq c_j
$$
and $y_i\geq 0$ for $i=1,\dots, 30$.

#### Describe its interpretation in the context of team formation.

In our dual problem the dual variables show us how the constraints affect the team building, that is the how much we sacrifice the optimal solution by adding constraints.

We begin by noting that at optimality the dual variables denote the shadow price for each contraint i.e. how the optimal value would change if the constraint was relaxed by one unit. But generally the dual variables can be split in 5 groups depending on their corresponding constraint.

- $ y_1-y_4 $ -> They represent the cost of ensuring that each personality group has one member i.e  the bigger they are the more the constraint is costing the optimal value. We saw this in 3d, $y_3$ ($y_2$ in 3 because of python indexing) had a value 0.04 which meant if we would relax this constraint by one we could increase the optimal value by 0.04.
- $y_5-y_{12}$ -> They represent the cost of keeping a balanced team.
- $y_{13}$ -> Represents the cost of keeping the team under 7 members. We saw that $y_{13}$ was very positive which means that the constraint was very restricting and increasing the team size would have given a better optimal value.
- $y_{14}$ -> Represents the cost of keeping the team under 5 members.
- $y_{15}-y_{30}$ -> Represents the cost of just using the avaialable members. We could for example see that if some $y$ is very big here, finding an extra candidate of the corresponding personality type could benefit the teams score.

All these variables tell us how the constraints we set affect the final team score. If they are positive the constraint is restricting and if they are zero additional flexibilty in the constraints have no value.

And based on the dual values we can determine if we want to relax or tighten constraints to better our results. Let's take a couple exxamples.

- y_{15} = 0.12 and $y_{15}$ is the dual variable corresponding to the constraint of available *ESFP* personalities. Because it is positive having an extra person of this peronality type would improve our benefit score by 0.12.
- y_{16} = 0 and $y_{16}$ is the dual variable corresponding to the constraint of available *ESFJ* personalities. Because it is 0e having an extra person of this peronality type would not improve our benefit score.
